In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install bertviz
!pip install plotly
!pip install pyyaml==5.4.1

     |████████████████████████████████| 4.0 MB 16.4 MB/s 
     |████████████████████████████████| 895 kB 71.1 MB/s 
     |████████████████████████████████| 596 kB 86.7 MB/s 
     |████████████████████████████████| 77 kB 8.5 MB/s 
     |████████████████████████████████| 6.6 MB 63.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 15.3 MB/s 
     |████████████████████████████████| 157 kB 15.9 MB/s 
     |████████████████████████████████| 132 kB 77.3 MB/s 
     |████████████████████████████████| 79 kB 9.6 MB/s 
     |████████████████████████████████| 8.7 MB 84.6 MB/s 
     |████████████████████████████████| 138 kB 91.7 MB/s 
     |████████████████████████████████| 127 kB 81.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import json
import torch
import pandas as pd
import numpy as np
torch.cuda.empty_cache()
from pathlib import Path # 파일시스템의 경로를 단순한 문자열이 아닌 객체로 다루게 하는 것
from torch.utils.data import Dataset, DataLoader, SequentialSampler

from sklearn.model_selection import train_test_split
from termcolor import colored
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim

from transformers import (
    AdamW,
    AutoModel,
    AutoTokenizer,
    AutoModelForQuestionAnswering
)
from tqdm.auto import tqdm
from bertviz import head_view

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import gc


import warnings

warnings.filterwarnings(action='ignore') 
from plotly import graph_objs as go

from collections import Counter
import plotly.express as px

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os

DIR = "/content/gdrive/MyDrive/k-digital-goorm-3-korean-mrc/"
TRAIN_SOURCE = os.path.join(DIR, "train.json")
TEST_SOURCE = os.path.join(DIR, "test.json")

with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())

with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())

In [ ]:
train = pd.DataFrame(columns=['guid' ,'title' ,'news_category', 'context', 'question' ,'answer_start', 'text'])
guid = 0
for data in TRAIN_DATA['data']:
    for paragraphs in data['paragraphs']:
        for line in paragraphs['qas']:
            train.loc[guid, 'context'] = paragraphs['context']
            train.loc[guid, 'question'] = line['question']
            train.loc[guid, 'answer_start'] = line['answers'][0]['answer_start']
            train.loc[guid, 'text'] = line['answers'][0]['text']            
            train.loc[guid, 'title'] = data['title']
            train.loc[guid, 'guid'] = line['guid']
            guid += 1

In [ ]:
test = pd.DataFrame(columns=['guid','title' ,'news_category', 'context', 'question' ])
guid = 0
for data in TEST_DATA['data']:
    for paragraphs in data['paragraphs']:
        for line in paragraphs['qas']:
            test.loc[guid, 'context'] = paragraphs['context']
            test.loc[guid, 'question'] = line['question']
            test.loc[guid, 'title'] = data['title']
            test.loc[guid, 'guid'] = line['guid']
            guid += 1

In [ ]:
train_ai_hub = pd.read_csv('/content/gdrive/MyDrive/k-digital-goorm-3-korean-mrc/ai_hub(도서).csv')

In [ ]:
len(train)

12037

In [ ]:
train = pd.concat([train, train_ai_hub]).reset_index().drop('index', axis=1)
train = train.reset_index().drop('index', axis=1)

In [ ]:
train = train[:50000]

In [ ]:
def get_answers(x):
    start = x[0]
    text = x[1]
    return {
        'answer_start': [start],
        'text': [text]
    }


train['answers'] = train[['answer_start', 'text']].apply(get_answers, axis=1)

In [ ]:
train_df, val_df = train_test_split(train, test_size=0.1, random_state=42)

train_df = train_df.reset_index().drop('index', axis=1)
val_df = val_df.reset_index().drop('index', axis=1)

train_df.shape, val_df.shape

((18000, 7), (2000, 7))

In [ ]:
max_length = 512

In [ ]:
MODEL_NAME = 'DHBaek/xlm-roberta-large-korquad-mask'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/720 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
train_df[:10]['answers']

0    {'answer_start': [584], 'text': ['성추행 의혹이 제기된 ...
1          {'answer_start': [349], 'text': ['인구 조사국']}
2             {'answer_start': [584], 'text': ['윤다인']}
3    {'answer_start': [483], 'text': ['“그 조건은 미국이 만...
4            {'answer_start': [101], 'text': ['카이스트']}
5            {'answer_start': [304], 'text': ['113명']}
6      {'answer_start': [752], 'text': ['미국을 비롯한 서방']}
7    {'answer_start': [563], 'text': ['"몸싸움 과정에서 흥분...
8             {'answer_start': [423], 'text': ['성상철']}
9           {'answer_start': [320], 'text': ['해양수산부']}
Name: answers, dtype: object

In [ ]:
def prepare_train_features(example, tokenizer):
    example["question"] = example["question"].lstrip()
    tokenized_example = tokenizer(
        example["question"],
        example["context"],
        truncation="only_second",
        max_length=max_length,
        stride=50, 
        return_overflowing_tokens=True, # 길이를 넘어가는 토큰들을 반환할 것인지
        return_offsets_mapping=True,  # 각 토큰에 대해 (char_start, char_end) 정보를 반환한 것인지
        padding="max_length"
    )

    sample_mapping = tokenized_example.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_example.pop("offset_mapping")

    features = []
    for i, offsets in enumerate(offset_mapping):
        feature = {}

        input_ids = tokenized_example["input_ids"][i]
        attention_mask = tokenized_example["attention_mask"][i]

        feature['input_ids'] = input_ids
        feature['attention_mask'] = attention_mask
        feature['offset_mapping'] = offsets

        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_example.sequence_ids(i)

        sample_index = sample_mapping[i]
        answers = example["answers"]

        if len(answers["answer_start"]) == 0:
            feature["start_position"] = cls_index
            feature["end_position"] = cls_index
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                feature["start_position"] = cls_index
                feature["end_position"] = cls_index
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                feature["start_position"] = token_start_index - 1
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                feature["end_position"] = token_end_index + 1

        features.append(feature)
    return features

In [ ]:
train_features, valid_features = [[] for _ in range(2)]
for i, row in tqdm(train_df.iterrows()):
    train_features += prepare_train_features(row, tokenizer)
for i, row in tqdm(val_df.iterrows()):
    valid_features += prepare_train_features(row, tokenizer)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
class DatasetRetriever(Dataset):
    def __init__(self, features, mode='train'):
        super(DatasetRetriever, self).__init__()
        self.features = features
        self.mode = mode
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, item):   
        feature = self.features[item]
        if self.mode == 'train':
            return {
                'input_ids':torch.tensor(feature['input_ids'] ,dtype=torch.long),
                'attention_mask':torch.tensor(feature['attention_mask'], dtype=torch.long),
                'offset_mapping':torch.tensor(feature['offset_mapping'], dtype=torch.long),
                'start_position':torch.tensor(feature['start_position'], dtype=torch.long),
                'end_position':torch.tensor(feature['end_position'], dtype=torch.long)
            }
        else:
            return {
                'input_ids':torch.tensor(feature['input_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(feature['attention_mask'], dtype=torch.long),
                'offset_mapping':feature['offset_mapping'],
                'sequence_ids':feature['sequence_ids'],
                'id':feature['example_id'],
                'context': feature['context'],
                'question': feature['question']
            }

In [ ]:
train_dataset = DatasetRetriever(train_features)
valid_dataset = DatasetRetriever(valid_features)

In [ ]:
import os
from statistics import mean

import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_
from transformers import AutoModelForQuestionAnswering
from transformers import default_data_collator

torch.manual_seed(42)
batch_size = 256
accumulation = 128
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)
model.cuda()
data_collator = default_data_collator

train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size//accumulation, 
    shuffle=True,
    collate_fn=data_collator,
    num_workers=2)

valid_loader = DataLoader(
    valid_dataset,
    batch_size=batch_size//accumulation, 
    shuffle=False, 
    collate_fn=data_collator, 
    num_workers=2)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# os.makedirs('dump', exist_ok=True)
train_losses = []
dev_losses = []

step = 0

for epoch in range(1, 11):
    print("Epoch", epoch)
    # Training
    running_loss = 0.
    losses = []
    model.train()
    progress_bar = tqdm(train_loader, desc='Train')
    for batch in progress_bar:
        del batch['offset_mapping']
        optimizer.zero_grad(set_to_none=True)
        batch = {key: value.cuda() for key, value in batch.items()}
        start = batch.pop('start_position')
        end = batch.pop('end_position')
        
        output = model(**batch)
        start_logits = output.start_logits
        end_logits = output.end_logits
        loss = F.cross_entropy(start_logits, start) + F.cross_entropy(end_logits, end)
        (loss / accumulation).backward()
        running_loss += loss.item()
        del batch, start, end, start_logits, end_logits, loss
        
        step += 1
        if step % accumulation:
            continue

        clip_grad_norm_(model.parameters(), max_norm=1.)
        optimizer.step()

        losses.append(running_loss / accumulation)
        running_loss = 0.
        progress_bar.set_description(f"Train - Loss: {losses[-1]:.3f}")
        wandb.log({"train loss": losses[-1]})
    train_losses.append(mean(losses))
    print(f"train score: {train_losses[-1]:.3f}")

    # Evaluation
    losses = []
    model.eval()
    for batch in tqdm(valid_loader, desc="Evaluation"):
        del batch['offset_mapping']
        batch = {key: value.cuda() for key, value in batch.items()}
        start = batch.pop('start_position')
        end = batch.pop('end_position')
        
        with torch.no_grad():
            output = model(**batch)
            start_logits = output.start_logits
            end_logits = output.end_logits
        loss = F.cross_entropy(start_logits, start) + F.cross_entropy(end_logits, end)

        losses.append(loss.item())
        wandb.log({"valid loss": losses[-1]})
        del batch, start, end, start_logits, end_logits, loss
    dev_losses.append(mean(losses))
    print(f"Evaluation score: {dev_losses[-1]:.3f}")

model.save_pretrained(f'/content/gdrive/MyDrive/k-digital-goorm-3-korean-mrc/dump/model(roberta_large, only_klue).{epoch}')

Downloading:   0%|          | 0.00/2.08G [00:00<?, ?B/s]

Epoch 1


Train:   0%|          | 0/14097 [00:00<?, ?it/s]

train score: 2.987


Evaluation:   0%|          | 0/1571 [00:00<?, ?it/s]

...

Evaluation score: 2.895
Epoch 3


Train:   0%|          | 0/14097 [00:00<?, ?it/s]

train score: 3.124


Evaluation:   0%|          | 0/1571 [00:00<?, ?it/s]

Evaluation score: 2.939


In [ ]:
gc.collect()

542

In [ ]:
def prepare_test_features(example, tokenizer):
    example["question"] = example["question"].lstrip()
    
    tokenized_example = tokenizer(
        example["question"],
        example["context"],
        truncation="only_second",
        max_length=512,
        stride=50,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    features = []
    for i in range(len(tokenized_example["input_ids"])):
        feature = {}
        feature["example_id"] = example['guid']
        feature['context'] = example['context']
        feature['question'] = example['question']
        feature['input_ids'] = tokenized_example['input_ids'][i]
        feature['attention_mask'] = tokenized_example['attention_mask'][i]
        feature['offset_mapping'] = tokenized_example['offset_mapping'][i]
        feature['sequence_ids'] = [0 if i is None else i for i in tokenized_example.sequence_ids(i)]
        features.append(feature)
    return features

In [ ]:
import collections

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 20):
    all_start_logits, all_end_logits = raw_predictions
    
    example_id_to_index = {k: i for i, k in enumerate(examples["guid"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()

    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    for example_index, example in examples.iterrows():
        feature_indices = features_per_example[example_index]

        min_null_score = None
        valid_answers = []
        
        context = example["context"]
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]

            sequence_ids = features[feature_index]["sequence_ids"]
            context_index = 1

            features[feature_index]["offset_mapping"] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(features[feature_index]["offset_mapping"])
            ]
            offset_mapping = features[feature_index]["offset_mapping"]
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
        
        predictions[example["guid"]] = best_answer["text"]
        
        
    return predictions

In [ ]:
test_features = []
for i, row in test.iterrows():
    test_features += prepare_test_features(row, tokenizer)

In [ ]:
test_dataset = DatasetRetriever(test_features, mode='test')
test_dataloader = DataLoader(
    test_dataset,
    batch_size=2, 
    shuffle=False,
    num_workers=2,
    pin_memory=True, 
    drop_last=False
)

In [ ]:
def get_predictions(trained_model):
    model = trained_model.cuda()
    
    start_logits = []
    end_logits = []
    model.eval()
    for batch in tqdm(test_dataloader):
        with torch.no_grad():
            output = model(batch['input_ids'].cuda(), batch['attention_mask'].cuda())
            start_logits.append(output.start_logits.cpu().numpy().tolist())
            end_logits.append(output.end_logits.cpu().numpy().tolist())
            del output
    gc.collect()
    return np.vstack(start_logits), np.vstack(end_logits)

In [ ]:
start_logits, end_logits = get_predictions(trained_model=model)
# Now Calling Function 

fin_preds = postprocess_qa_predictions(test, test_features, (start_logits, end_logits))

  0%|          | 0/3265 [00:00<?, ?it/s]

Post-processing 4008 example predictions split into 6530 features.


In [ ]:
from string import punctuation
submission = []
for p1, p2 in fin_preds.items():
    p2 = " ".join(p2.split())
    p2 = p2.strip(punctuation)
    submission.append((p1, p2))
    
sample = pd.DataFrame(submission, columns=["id", "Predicted"])

#test_data =pd.merge(left=test_df,right=sample,on='id')

In [ ]:
sample.to_csv('/content/gdrive/MyDrive/k-digital-goorm-3-korean-mrc/out/roberta_v2(klue+aihub).csv', index=False)
sample = sample.rename({'id':'guid'}, axis='columns')

test_data =pd.merge(left=test,right=sample,on='guid')